# Buy Me A Coffee Support Scraper
Extract support data and create a markdown table

In [1]:
import re
from time import sleep
import requests
from bs4 import BeautifulSoup

In [2]:
url = "https://www.buymeacoffee.com/PySimpleGUI"

headers = {
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "accept-encoding": "gzip, deflate, br",
    "accept-language": "en-US,en;q=0.9",
    "cache-control": "max-age=0",
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "none",
    "sec-fetch-user": "?1",
    "upgrade-insecure-requests": "1",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36 Edg/86.0.622.51"
}

params = {
    "page":0,
    "notification":1
}

In [3]:
def main(url, headers, params):
    """Get card data from page"""
    page_cards = []
    
    # create persistant session
    session = requests.Session()
    response = session.get(url, headers=headers)

    # get first page of cards
    soup = BeautifulSoup(response.text, 'html.parser')
    cards = soup.find_all('div', 'donation-sec')

    # collect all card data
    for card in cards:
        image = card.find('img').get('data-src')
        user = card.find('img').get('alt')
        coffee = card.find(text=re.compile(r'bought'))
        page_cards.append((image, user, coffee))    

    # collect all remaining cards
    while True:
        sleep(1)
        params['page'] += 1
        response = session.get(url, headers=headers, params=params)
        
        # check to see if there are any cards
        if not response.text:
            break
        
        # get next page of cards
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('div', 'donation-sec')

        # collect all card data
        for card in cards:
            image = card.find('img').get('data-src')
            user = (card.find('img').get('alt').strip() or "Someone")
            coffee = card.find(text=re.compile(r'bought'))
            page_cards.append((image, user, coffee))
            
    return page_cards

In [4]:
def create_md_table(donations):
    """Generate markdown table form donations data"""
    table = """|  | user | support |  \n|---|---|---|  \n"""
    for row in donations:
        table += '| <img src="{}" width=25> | {} | {} |  \n'.format(*row)
    return table

In [5]:
# run main program
donations = main(url, headers, params)

# generate markdown table
md_table = create_md_table(donations)

In [6]:
print(md_table)

|  | user | support |  
|---|---|---|  
| <img src="https://img.buymeacoffee.com/api/?name=%40ktrinko&size=300&bg-image=bmc&background=5F7FFF" width=25> | @ktrinko | bought 3 coffee |  
| <img src="https://img.buymeacoffee.com/api/?url=aHR0cHM6Ly9jZG4uYnV5bWVhY29mZmVlLmNvbS91cGxvYWRzL3Byb2ZpbGVfcGljdHVyZXMvMjAyMC8wMS9jNGYwNmUxMThlNTg5MTgyOWZlZjMxNWRmZWVlMzgyMS5qcGc=&size=150&name=Paolo+Amoroso" width=25> | Paolo Amoroso | bought a coffee |  
| <img src="https://img.buymeacoffee.com/api/?name=Someone&size=300&bg-image=bmc&background=5F7FFF" width=25> |  | bought 5 coffee |  
| <img src="https://img.buymeacoffee.com/api/?name=%40RYRJB&size=300&bg-image=bmc&background=5F7FFF" width=25> | @RYRJB | bought 5 coffee |  
| <img src="https://img.buymeacoffee.com/api/?name=Thiagojm&size=300&bg-image=bmc&background=5F7FFF" width=25> | Thiagojm | bought 5 coffee |  
| <img src="https://img.buymeacoffee.com/api/?name=%40tuakau66&size=300&bg-image=bmc&background=5F7FFF" width=25> | @tuakau66 | bough

### Insert the markdown text into a "markdown" cell

|  | user | support |  
|---|---|---|  
| <img src="https://img.buymeacoffee.com/api/?name=%40ktrinko&size=300&bg-image=bmc&background=5F7FFF" width=25> | @ktrinko | bought 3 coffee |  
| <img src="https://img.buymeacoffee.com/api/?url=aHR0cHM6Ly9jZG4uYnV5bWVhY29mZmVlLmNvbS91cGxvYWRzL3Byb2ZpbGVfcGljdHVyZXMvMjAyMC8wMS9jNGYwNmUxMThlNTg5MTgyOWZlZjMxNWRmZWVlMzgyMS5qcGc=&size=150&name=Paolo+Amoroso" width=25> | Paolo Amoroso | bought a coffee |  
| <img src="https://img.buymeacoffee.com/api/?name=Someone&size=300&bg-image=bmc&background=5F7FFF" width=25> |  | bought 5 coffee |  
| <img src="https://img.buymeacoffee.com/api/?name=%40RYRJB&size=300&bg-image=bmc&background=5F7FFF" width=25> | @RYRJB | bought 5 coffee |  
| <img src="https://img.buymeacoffee.com/api/?name=Thiagojm&size=300&bg-image=bmc&background=5F7FFF" width=25> | Thiagojm | bought 5 coffee |  
| <img src="https://img.buymeacoffee.com/api/?name=%40tuakau66&size=300&bg-image=bmc&background=5F7FFF" width=25> | @tuakau66 | bought 3 coffee |  
| <img src="https://img.buymeacoffee.com/api/?name=Cheong+Koo&size=300&bg-image=bmc&background=5F7FFF" width=25> | Cheong Koo | bought 3 coffee |  
| <img src="https://img.buymeacoffee.com/api/?name=Miodrag+Mili%C4%8Devi%C4%87&size=300&bg-image=bmc&background=5F7FFF" width=25> | Miodrag Miličević | bought 15 coffee |  
| <img src="https://img.buymeacoffee.com/api/?name=Someone&size=300&bg-image=bmc&background=5F7FFF" width=25> | Someone | bought 7 coffee |  
| <img src="https://img.buymeacoffee.com/api/?name=Laura+Becker&size=300&bg-image=bmc&background=5F7FFF" width=25> | Laura Becker | bought 5 coffee |  
| <img src="https://img.buymeacoffee.com/api/?name=%40rikhuygen&size=300&bg-image=bmc&background=5F7FFF" width=25> | @rikhuygen | bought 3 coffee |  
| <img src="https://img.buymeacoffee.com/api/?name=jeff%40puffergas.com&size=300&bg-image=bmc&background=5F7FFF" width=25> | jeff@puffergas.com | bought 5 coffee |  
| <img src="https://img.buymeacoffee.com/api/?name=Mike&size=300&bg-image=bmc&background=5F7FFF" width=25> | Mike | bought a coffee |  